In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import os

In [ ]:
filenames = os.listdir('data')

for i in range(len(filenames)):
    filenames[i] = unicodedata.normalize('NFKC', filenames[i].split('.')[0])

In [ ]:
df_keyword = []

df = pd.DataFrame()
for filename in filenames:
    temp1 = pd.read_csv('./data/'+filename+'.csv')
    df = pd.concat([df, temp1])
    temp2 = pd.read_csv('./data_plus/'+filename+'.csv')
    df = pd.concat([df, temp2])
    df_keyword.append(list(pd.concat([temp1, temp2]).content))

In [ ]:
for i in range(10):
    for j in range(len(df_keyword[i])):
        df_keyword[i][j] = unicodedata.normalize('NFKC', df_keyword[i][j])

In [ ]:
content_lengths = [len(content) for content in list(df.content)]

In [ ]:
df['len'] = content_lengths

In [ ]:
df = df.sort_values('len')

In [ ]:
df = df.drop_duplicates('content')

In [ ]:
df.describe()

In [ ]:
df = df[(df.len>100) & (df.len<3000)]

In [ ]:
df.describe()

In [ ]:
new_label = []

for item in df.itertuples():
    word_count = [0 for _ in range(10)]
    title = unicodedata.normalize('NFKC', item.title)
    content = unicodedata.normalize('NFKC', item.content)

    if content in df_keyword[3]:
        new_label.append(filenames[3])
        continue

    for idx, word in enumerate(filenames):
        word_count[idx] = 1.2 * title.count(word) + content.count(word)
        
    if sum(word_count)>0:
        new_label.append(filenames[np.argmax(word_count)])
    else:
        candidate = []
        
        for i in range(10):
            if content in df_keyword[i]:
                candidate.append(i)
        
        new_label.append(filenames[candidate[np.random.randint(len(candidate))]])

In [ ]:
from collections import Counter
counter=Counter(new_label)

In [ ]:
counter

In [ ]:
df['label'] = new_label

In [ ]:
df = df.drop(['len', 'keyword'], axis=1)

In [ ]:
df.to_csv('종합 데이터.csv', index=False)

In [ ]:
d1 = pd.read_csv('종합 데이터.csv')
summ = list(pd.read_csv('summ.csv').summ)

d1['summarization'] = summ

d1.to_csv('요약 데이터.csv', index=False)